In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tqdm
from tensorflow.keras.utils import Sequence, to_categorical
from keras.utils.np_utils import to_categorical
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Dropout, BatchNormalization, UpSampling2D, InputLayer
from tensorflow.keras.layers import Input
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import average_precision_score

## __Load dữ liệu train, test, val__

In [ ]:
# make shorcut cho folder CS114-ML trước
%cd "/content/gdrive/MyDrive/CS114-ML/data_ndarray" 

/content/gdrive/.shortcut-targets-by-id/12IVdVQOkY2icQgpyJaH6y16BPdip3RKW/CS114-ML/data_ndarray


In [ ]:
with open('X.npy', 'rb') as f:
    X = np.load(f)
with open('y.npy', 'rb') as f:
    y = np.load(f)

## __Cài đặt mô hình__

In [ ]:
df = pd.DataFrame(columns=["model", "fold_1", "fold_2", "fold_3", "fold_4", "mWAP", "std"])

In [ ]:
# khai báo số lượng class và shape ảnh training
num_classes = y.shape[1]
input_shape = X.shape[1:]

In [ ]:
# training config
base_learning_rate = 0.0001
epochs = 25
batch_size = 64

checkpoint_path = "/content/gdrive/MyDrive/CS114-ML/checkpoint/densenet"

callbacks = ModelCheckpoint(
        filepath=os.path.join(checkpoint_path, "densenet-{epoch}.h5"),
        save_best_only=True, 
        monitor="val_loss",
        verbose=1)

In [ ]:
def get_model():
  # khai báo mô hình CNN sử dụng
  base_model = tf.keras.applications.DenseNet121(include_top=False, weights="imagenet", input_shape=input_shape)

  # đóng băng (k cập nhật trọng số khi training) toàn bộ base_model
  base_model.trainable = False

  input_layer = tf.keras.Input(shape=input_shape) # khởi tạo input layer

  rescale = tf.keras.layers.Rescaling(1./255) 
  x = rescale(input_layer) # scale ảnh từ [0, 255] về [0, 1]
  x = base_model(x, training=False) # đưa kết quả input_layer vào base_model
  x = tf.keras.layers.GlobalAveragePooling2D()(x) # "flatten" kết quả đầu ra của base_model thành vector 1 chiều

  output_layer = tf.keras.layers.Dense(num_classes, activation="sigmoid")(x) # khởi tạo output layer - dự đoán toàn bộ 

  model = Model(inputs=input_layer, outputs=output_layer) # ghép input layer và output layer lại thành 1 model hoàn chỉnh

  model.compile(loss='binary_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              metrics=['accuracy'])
  
  return model


In [ ]:
def evaluate_model(X, y):
  ap_results = [] # khai báo biến chứa kết quả ap của từng fold
  
  cv = RepeatedKFold(n_splits=4, n_repeats=1, random_state=1)
  
  for train_ix, test_ix in cv.split(X):
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    print("1")

    model = get_model()
    print("2")
    model.fit(X_train, y_train, verbose=1, epochs=epochs)

    y_pred_scores = model.predict(X_test, batch_size=batch_size, verbose=1)
    y_pred_scores = np.around(y_pred_scores, decimals=3)

    weighted_ap = average_precision_score(y_test, y_pred_scores, average='weighted')

    print('>%.3f' % weighted_ap)
    ap_results.append(weighted_ap)

  mwap = np.round(np.mean(ap_results), 3)
  std = np.round(np.std(ap_results), 3)
  print('mAP: %.3f (%.3f)' % (np.mean(ap_results), np.std(ap_results)))
  ap_results = np.around(ap_results, decimals=3)
  df.loc[0] = ["dense_net"] + [ap_results[0]] + [ap_results[1]] + [ap_results[2]] + [ap_results[3]] + [mwap] + [std]
  return ap_results

In [ ]:
results = evaluate_model(X, y)

1
29097984/29084464 [==============================] - 0s 0us/step
2
Epoch 1/25
95/95 [==============================] - 19s 64ms/step - loss: 0.4211 - accuracy: 0.0260
Epoch 2/25
95/95 [==============================] - 6s 59ms/step - loss: 0.2441 - accuracy: 0.0721
Epoch 3/25
95/95 [==============================] - 6s 59ms/step - loss: 0.2187 - accuracy: 0.0965
Epoch 4/25
95/95 [==============================] - 6s 59ms/step - loss: 0.2046 - accuracy: 0.1287
Epoch 5/25
95/95 [==============================] - 6s 59ms/step - loss: 0.1931 - accuracy: 0.1564
Epoch 6/25
95/95 [==============================] - 6s 59ms/step - loss: 0.1829 - accuracy: 0.1906
Epoch 7/25
95/95 [==============================] - 6s 59ms/step - loss: 0.1737 - accuracy: 0.2160
Epoch 8/25
95/95 [==============================] - 6s 59ms/step - loss: 0.1653 - accuracy: 0.2473
Epoch 9/25
95/95 [==============================] - 6s 59ms/step - loss: 0.1577 - accuracy: 0.2776
Epoch 10/25
95/95 [====================

In [ ]:
file_name = "dense_net_evaluation.csv"
save_path = os.path.join(checkpoint_path, file_name)
df.to_csv(save_path)